In [26]:
import time
import datetime
s = "2022-11-09"
time.mktime(datetime.datetime.strptime(s, "%Y-%m-%d").timetuple())

1667948400.0

In [7]:
!pip install cassandra-driver

In [3]:
#import modules
from cassandra.cluster import Cluster
import pandas as pd

In [4]:
#connect to the cluster
cluster = Cluster(['127.0.0.1'],port=9042)

In [5]:
#connect to the keyspace
session = cluster.connect('DB')

In [6]:
#get the data
df = pd.DataFrame(list(session.execute('SELECT * FROM "Card";')))

In [5]:
#show data
df

,id_card,end_of_validity,id_client,type
0,1,2022-11-05,1,month
1,2,2022-11-05,2,month
2,3,2022-11-05,3,year


In [46]:
{
    "day":"03/11/2022",
    "new_subscibers": {
        "number":30,
        "normal_user":5,
        "monthly":15,
        "year":10
        
    }
}


{
    "day":"03/11/2022",
    "interval_start":8,
    "interval_end":9,
    "tram_A":{
        "users":30,
        "station1":10,
        "station2":10,
        "station3":10,
    },
    "tram_B":{
        "users":50,
        "station1":10,
        "station2":30,
        "station3":10,
    },
    "tram_C":{
        "users":100,
        "station1":40,
        "station2":50,
        "station3":10,
    }
}

{'day': '03/11/2022',
 'interval_start': 9,
 'tram_A': {'users': 30, 'station1': 10, 'station2': 10, 'station3': 10},
 'tram_B': {'users': 50, 'station1': 10, 'station2': 30, 'station3': 10},
 'tram_C': {'users': 100, 'station1': 40, 'station2': 50, 'station3': 10}}

In [51]:
#compute the number of new users
#get the last day for which the data was processed
last_day = df = pd.DataFrame(list(session.execute('SELECT max(day) as last_day FROM "statistics_1";')))['last_day'][0]

In [53]:
last_day == None

True

In [8]:
from datetime import datetime
from datetime import timedelta
import pandas as pd
import time

In [120]:
def statistics_1(session):
    last_day = pd.DataFrame(list(session.execute('SELECT max(day) as last_day FROM "statistics_1";')))['last_day'][0]
    #print(last_day)
    if last_day != None:
        #get the users for the new day
        date_1 = datetime.strptime(str(last_day), "%Y-%m-%d")
        end_date = date_1 + timedelta(days=1)
        #end_date = time.mktime(datetime.datetime.strptime(end_date, "%Y/%m/%d").timetuple())

        dates = pd.DataFrame(list(session.execute(f"SELECT toDate(timestamp) as date\
         FROM \"Event\" where timestamp >= '{end_date}' allow filtering;")))['date'].unique()
        
    else:
        dates = pd.DataFrame(list(session.execute(f'SELECT toDate(timestamp) as date\
         FROM "Event";')))['date'].unique()


    dates.sort()
    
    for date in dates:

            date_time_obj = datetime. strptime(str(date), '%Y-%m-%d')

            if (date_time_obj.date() >= datetime.now().date()):
                print(f"you have to wait until next day to process {date_time_obj.date()} data")
                return
            #print(date)
            #date = datetime.fromtimestamp(date).date()
            date_1 = (datetime.strptime(str(date), "%Y-%m-%d"))
            end_date = date_1 + timedelta(days=1)
            end_date = str(end_date.date())
            date = str(date)
            #print(end_date.date())
            #end_date = int(time.mktime(datetime.strptime(str(end_date.date()), "%Y-%m-%d").timetuple()))
            #start =  int(time.mktime(datetime.strptime(str(date), "%Y-%m-%d").timetuple()))
            #print(int(start))
            #print(int(end_date))
            #print(date)
            #print(end_date)
            users =  pd.DataFrame(list(session.execute(f"SELECT id_user FROM \"Event\" where \
            timestamp >= '{date}' and  timestamp < '{end_date}' allow filtering;")))
            #print(users)
            
            users = users['id_user'].unique()
            
            #get the old users 

            old_users =  pd.DataFrame(list(session.execute(f"SELECT id_user FROM \"Event\" \
            where timestamp < '{date}' ALLOW FILTERING;")))

            try:
                old_users = old_users['id_user'].unique()
            except:
                old_users = []

            #find the new users
            #new_users_count = 0
            new_users = []
            for user in users:
                if user not in old_users:
                    new_users.append(user)


            details = {"month":[],"year":[]}

            #find the type of subsciption for new users
            prepared_statement = session.prepare('SELECT type from "Card" where id_client = ? ALLOW FILTERING;')

            for user in new_users:
                card_type = pd.DataFrame(list(session.execute(prepared_statement,[user])))['type'][0]
                if(card_type == "month"):
                    details['month'].append(user)
                elif(card_type == "year"):
                    details['year'].append(user)
            
            session.execute(f"INSERT INTO statistics_1(day, month_user, new_subs, year_user)\
            VALUES ('{date}', {len(details['month'])},{len(new_users)} ,{len(details['year'])} );")
            print("statisctics saved!")
                    

In [128]:
def statistics_2(session):
    last_day = df = pd.DataFrame(list(session.execute('SELECT max(day) as last_day FROM "statistics_2";')))['last_day'][0]
    if last_day != None:
        #get the users for the new day
        date_1 = datetime.strptime(str(last_day), "%Y-%m-%d")
        end_date = date_1 + timedelta(days=1)
        #end_date = time.mktime(datetime.datetime.strptime(end_date, "%Y/%m/%d").timetuple())

        try:

            dates = pd.DataFrame(list(session.execute(f"SELECT toDate(timestamp) as date\
            FROM \"Event\" where timestamp >= '{end_date}' allow filtering;")))['date'].unique()
        except:
            print("No new data to process !!")
            return
        
    else:
        dates = pd.DataFrame(list(session.execute(f'SELECT toDate(timestamp) as date\
         FROM "Event";')))['date'].unique()
    #print(dates)
    dates.sort()
    for date in dates:
        date_time_obj = datetime. strptime(str(date), '%Y-%m-%d')
        if (date_time_obj.date() >= datetime.now().date()):
                print(f"you have to wait until next day to process {date_time_obj.date()} data")
                return
        date_1 = (datetime.strptime(str(date), "%Y-%m-%d"))
        end_date = date_1 + timedelta(days=1)
        end_date = str(end_date.date())
        date = str(date)
        """ date_1 = datetime.datetime.strptime(date, "%Y-%m-%d")
        end_date = date_1 + timedelta(days=1)
        end_date = time.mktime(datetime.datetime.strptime(end_date, "%Y-%m-%d").timetuple())

        start =  time.mktime(datetime.datetime.strptime(date, "%Y-%m-%d").timetuple()) """
    
        data = pd.DataFrame(list(session.execute(f"SELECT id_event, end_station, \"id_card \",\
         id_user, line, start_station, \"timestamp\", toDate(\"timestamp\") as day FROM \"Event\"\
          where timestamp >= '{date}' and timestamp < '{end_date}' allow filtering;")))

        for i in range(data.shape[0]):
            date = data.loc[i,'timestamp']
            data.loc[i,'Hour'] = str(date.time().hour)


        #get start_hour
        hours = ["00","01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23"]
        lines = ['A','B','C']
        for index in range(len(hours)):
                    start_hour = hours[index]
                    try:
                        end_hour = hours[index+1]
                    
                    except:
                        end_hour = "00"
                    #print(f"{start_hour} ---> {end_hour}")
                    df = data[(data['day'] == date) & (data['Hour']==start_hour)]

                    tram_lines = {}

                    for line in lines:
                        tram_lines[line] = {}
                        data_for_line = df[df['line'] == line]

                        tram_lines[line]['users'] = data_for_line.shape[0]

                        for station in data_for_line['start_station'].unique():
                            #print(station)

                            data_for_station = data_for_line[data_for_line['start_station'] == station]

                            tram_lines[line][station] = data_for_station.shape[0]
                    #print(tram_lines)
                    
        
        #write result in cassandra
                    print("ok")
                    max_id = pd.DataFrame(list(session.execute(f'SELECT max(id) as max\
         FROM "statistics_2";')))[max].unique()[0]
                    if(max_id == None):
                        max_id=1
                    else:
                        max_id = max_id+1

                    session.execute_async(f"INSERT INTO statistics_2(id, day, end_hour, start_hour,\"tram_A\", \"tram_B\", \"tram_C\")\
VALUES ({max_id},'{date.date()}','{start_hour}','{end_hour}',{tram_lines['A']} , {tram_lines['B']},{tram_lines['C']});")

        print("statistics are saved!!")






                


           
                    

In [97]:
{
    "day":"03/11/2022",
    "interval_start":8,
    "interval_end":9,
    "tram_A":{
        "users":30,
        "station1":10,
        "station2":10,
        "station3":10,
    },
    "tram_B":{
        "users":50,
        "station1":10,
        "station2":30,
        "station3":10,
    },
    "tram_C":{
        "users":100,
        "station1":40,
        "station2":50,
        "station3":10,
    }
}

{'day': '03/11/2022',
 'interval_start': 8,
 'interval_end': 9,
 'tram_A': {'users': 30, 'station1': 10, 'station2': 10, 'station3': 10},
 'tram_B': {'users': 50, 'station1': 10, 'station2': 30, 'station3': 10},
 'tram_C': {'users': 100, 'station1': 40, 'station2': 50, 'station3': 10}}

In [13]:
objectdate = datetime.fromtimestamp(1667692800).date()

In [14]:
hour = str(objectdate)
print(hour)

2022-11-06


In [93]:
dt = datetime( 1970, 1, 20,0,0,0)
timestamp = int( dt.timestamp() )
print(timestamp)

1638000


In [123]:
statistics_1(session)

you have to wait until next day to process 2022-11-07 data


In [129]:
statistics_2(session)

you have to wait until next day to process 2022-11-07 data


In [167]:
users =  pd.DataFrame(list(session.execute(f'SELECT id_user FROM "Event" \
            where request_timestamp  >= {int(1667689200)} and request_timestamp < {int(1667775600)} ALLOW FILTERING;')))

In [168]:
def statistics_1(session):
    last_day = pd.DataFrame(list(session.execute('SELECT max(day) as last_day FROM "statistics_1";')))['last_day'][0]
    if last_day != None:
        #get the users for the new day
        date_1 = datetime.strptime(last_day, "%m/%d/%y")
        end_date = date_1 + timedelta(days=1)

        end_date = time.mktime(datetime.datetime.strptime(end_date, "%Y/%m/%d").timetuple())

        dates = pd.DataFrame(list(session.execute(f'SELECT distinct CAST(totimestamp(mintimeuuid(request_timestamp)) as date) as day\
         FROM "Event" where request_timestamp >= {end_date};')))['day'].unique()
        
    else:
        dates = pd.DataFrame(list(session.execute(f'SELECT CAST(totimestamp(mintimeuuid(request_timestamp)) as date) as day\
         FROM "Event";')))['day'].unique()

    
    for date in dates:
            print(date)
            date_1 = datetime.strptime(str(date), "%Y-%m-%d")
            end_date = date_1 + timedelta(days=1)
            print(end_date.date())
            end_date = time.mktime(datetime.strptime(str(end_date.date()), "%Y-%m-%d").timetuple())

            start =  time.mktime(datetime.strptime(str(date), "%Y-%m-%d").timetuple())
            print(int(start))
            print(int(end_date))
            users =  pd.DataFrame(list(session.execute(f'SELECT id_user FROM "Event" \
            where request_timestamp  >= {int(start)} and request_timestamp < {int(end_date)} ALLOW FILTERING;')))
            print(users)
            users = users['id_user'].unique()
            
            #get the old users 

            old_users =  pd.DataFrame(list(session.execute(f'SELECT id_user FROM "Event" \
            where request_timestamp < {start};')))

            old_users = old_users['id_user'].unique()

            #find the new users
            #new_users_count = 0
            new_users = []
            for user in users:
                if user not in old_users:
                    new_users.append(user)


            details = {"month":[],"year":[]}

            #find the type of subsciption for new users
            prepared_statement = session.prepare('SELECT type from Card where id_client = ?')

            for user in new_users:
                card_type = pd.DataFrame(list(session.execute(prepared_statement,[user])))['type'][0]
                if(card_type == "month"):
                    details['month'].append(user)
                elif(card_type == "year"):
                    details['year'].append(user)

            session.execute_async(f'INSERT INTO statistics_1(day, month_user, new_subs, year_user)\
            VALUES ({date}, {len(details["month"])},{len(new_users)} ,{len(details["year"])} );')

""


In [176]:
import pytz
user_tz = pytz.timezone('US/Central')
timestamp_naive = datetime.datetime(2022,11,6) 
timestamp_utc = pytz.utc.localize(timestamp_naive)
timestamp_presented = timestamp_utc.astimezone(user_tz)

In [181]:
print(timestamp_presented)

2022-11-05 19:00:00-05:00


In [17]:
df = pd.DataFrame(list(session.execute(f'SELECT toDate(request_timestamp) as day\
         FROM "Event";')))

In [18]:
df

,day
0,1970-01-20


In [77]:
users =  pd.DataFrame(list(session.execute(f"SELECT id_user FROM \"Event\" where timestamp >= '2022-11-06' and  timestamp < '2022-11-07' allow filtering;")))

In [78]:
users

,id_user
0,1
1,2
2,3


In [91]:
print(dates['date'])

0    2022-11-06
1    2022-11-06
2    2022-11-06
Name: date, dtype: object


In [87]:
dates = pd.DataFrame(list(session.execute(f'SELECT toDate(timestamp) as date\
         FROM "Event";')))#['date'].unique()

In [77]:
max_id = pd.DataFrame(list(session.execute(f'SELECT max(id) as max\
         FROM "statistics_2";')))[max].unique()

In [101]:

date_time_obj = datetime. strptime("2022-11-06", '%Y-%m-%d')


date_time_obj.date() < datetime.now().date()

True

In [ ]:
def processOrNot(date):


    #